# Notes

Collaborative filters can further be classified into two types:

* `user-based filtering` - recommend products to a user that **similar** users have liked. If tony and jeris have a similar in games, and tony plays a new game and likes it, it is highly likely that jeris will like it to.

* `item-based filtering` - extremely similar to the content recommendation engine that you built. These systems identify similar items based on how people have rated it in the past. If tony and jeris gave 5 stars to LOTR and the Hobbit, the system identifies the items as similar, so if lehcar buys LOTR, the system will recommend the Hobbit.